# Tutorial 18: Azure API Management for MCP Servers

## Enterprise API Gateway for MCP Architecture

### What You'll Learn

In this tutorial, you'll expose the **FastMCP Travel Server** (deployed in Tutorial 16) through **Azure API Management (APIM)**, adding enterprise-grade security, monitoring, and governance.

**Key Concepts:**
- Exposing MCP servers through APIM
- API Key authentication
- Rate limiting and throttling
- Monitoring and analytics
- Testing MCP through APIM gateway

**Using Existing Resources:**
- **APIM**: `apim-a35tm-aiagents` (already deployed)
- **MCP Server**: `travel-mcp-server` Container App (from Tutorial 16)

**Duration:** 30 minutes

---

### Prerequisites

- Completed Tutorial 16 (FastMCP deployed to Container Apps)
- Azure CLI installed and logged in
- Existing APIM instance

## Part 1: Understanding Azure API Management for MCP

### What is Azure API Management?

**Azure API Management (APIM)** is a fully managed service that enables organizations to publish, secure, transform, maintain, and monitor APIs. For MCP servers, APIM provides:

- **Unified Gateway**: Single entry point for all MCP servers
- **Security**: Authentication, authorization, rate limiting
- **Monitoring**: Analytics, logging, tracing
- **Governance**: Versioning, policies, documentation
- **Developer Portal**: Self-service API discovery

### Why Use APIM with MCP Servers?

**Benefits:**
1. **Centralized Security**: Apply consistent security policies across all MCP servers
2. **Rate Limiting**: Prevent abuse and manage costs
3. **Monitoring**: Track usage, performance, and errors
4. **Versioning**: Manage multiple versions of MCP tools
5. **Multi-Region**: Deploy to multiple Azure regions
6. **Developer Experience**: Portal for discovering and testing MCP tools

### Architecture: APIM + Multiple MCP Servers

```
┌──────────────────────────────────────────────────────┐
│   MCP Clients (VS Code, Agents, Copilot)            │
└────────────────────┬─────────────────────────────────┘
                     │
                     │ HTTPS + OAuth
                     ↓
┌──────────────────────────────────────────────────────┐
│   Azure API Management                               │
│   ┌──────────────────────────────────────────────┐   │
│   │  API Gateway                                 │   │
│   │  ├─ Authentication (OAuth, API Key)          │   │
│   │  ├─ Rate Limiting (100 calls/min)            │   │
│   │  ├─ IP Filtering                             │   │
│   │  ├─ Request/Response Transformation          │   │
│   │  ├─ Logging & Monitoring                     │   │
│   │  └─ Caching                                  │   │
│   └──────────────────────────────────────────────┘   │
└─────────────┬────────────────┬───────────────────────┘
              │                │
              ↓                ↓
┌──────────────────────┐  ┌──────────────────────┐
│  FastMCP Server      │  │  Logic Apps MCP      │
│  (Container Apps)    │  │  (Standard)          │
│  - Flight Search     │  │  - Send Email        │
│  - Hotel Booking     │  │  - Calendar          │
│  - Currency Convert  │  │  - SharePoint        │
└──────────────────────┘  └──────────────────────┘
```

## Part 2: Creating Azure API Management Instance

Let's create an APIM instance to host our MCP servers.

In [ ]:
# Configuration - Load from environment variables
import os
from dotenv import load_dotenv

load_dotenv()

# Azure Subscription
AZURE_SUBSCRIPTION_ID = os.getenv("AZURE_SUBSCRIPTION_ID")

# APIM Configuration (from .env)
APIM_NAME = os.getenv("APIM_NAME")
APIM_RESOURCE_GROUP = os.getenv("APIM_RESOURCE_GROUP")
APIM_GATEWAY_URL = os.getenv("APIM_GATEWAY_URL")

# MCP Server Configuration (from Tutorial 16)
MCP_SERVER_NAME = os.getenv("MCP_SERVER_NAME")
MCP_SERVER_RG = os.getenv("MCP_SERVER_RG")
MCP_BACKEND_URL = os.getenv("MCP_BACKEND_URL")

# Validate required environment variables
required_vars = {
    "AZURE_SUBSCRIPTION_ID": AZURE_SUBSCRIPTION_ID,
    "APIM_NAME": APIM_NAME,
    "APIM_RESOURCE_GROUP": APIM_RESOURCE_GROUP,
    "APIM_GATEWAY_URL": APIM_GATEWAY_URL,
    "MCP_SERVER_NAME": MCP_SERVER_NAME,
    "MCP_SERVER_RG": MCP_SERVER_RG,
    "MCP_BACKEND_URL": MCP_BACKEND_URL,
}

missing_vars = [k for k, v in required_vars.items() if not v]
if missing_vars:
    print(f"⚠️  Missing environment variables: {', '.join(missing_vars)}")
    print("   Please add them to your .env file. See .env.example for reference.")
else:
    print("=" * 70)
    print("Azure API Management + MCP Server Configuration")
    print("=" * 70)
    print(f"""
  APIM Instance:
    Name:           {APIM_NAME}
    Resource Group: {APIM_RESOURCE_GROUP}
    Gateway URL:    {APIM_GATEWAY_URL}
    
  MCP Backend (from Tutorial 16):
    Name:           {MCP_SERVER_NAME}
    Backend URL:    {MCP_BACKEND_URL}
""")
    print("=" * 70)

In [ ]:
# Verify both resources exist and are running
import subprocess
import json
import os

# Set up PATH to include Azure CLI location
# This is needed because Jupyter's subprocess doesn't inherit the shell's PATH
AZ_PATH = "/opt/homebrew/opt/azure-cli/bin"
env = os.environ.copy()
env["PATH"] = f"{AZ_PATH}:{env.get('PATH', '')}"

def run_az_command(cmd):
    """Run Azure CLI command and return JSON output"""
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True, env=env)
    if result.returncode != 0:
        print(f"Error: {result.stderr}")
        return None
    return json.loads(result.stdout) if result.stdout.strip() else None

print("=" * 70)
print("Verifying Azure Resources")
print("=" * 70)

# Check APIM
print("\n[1] Checking APIM instance...")
apim_info = run_az_command(f'az apim show --name {APIM_NAME} --resource-group {APIM_RESOURCE_GROUP} --query "{{name: name, gatewayUrl: gatewayUrl, sku: sku.name}}"')
if apim_info:
    print(f"    ✅ APIM: {apim_info['name']}")
    print(f"       Gateway: {apim_info['gatewayUrl']}")
    print(f"       SKU: {apim_info['sku']}")
else:
    print("    ❌ APIM not found!")

# Check Container App
print("\n[2] Checking MCP Server (Container App)...")
ca_info = run_az_command(f'az containerapp show --name {MCP_SERVER_NAME} --resource-group {MCP_SERVER_RG} --query "{{name: name, fqdn: properties.configuration.ingress.fqdn, status: properties.runningStatus}}"')
if ca_info:
    print(f"    ✅ Container App: {ca_info['name']}")
    print(f"       FQDN: {ca_info['fqdn']}")
    print(f"       Status: {ca_info['status']}")
else:
    print("    ❌ Container App not found!")

print("\n" + "=" * 70)

## Part 2: Creating the MCP API in APIM

We'll create an API in APIM that routes to our FastMCP server. This involves:

1. **Creating a Backend** - Points to the Container App
2. **Creating an API** - Defines the /mcp path
3. **Creating Operations** - Defines the MCP endpoints (POST for messages)
4. **Applying Policies** - Security, rate limiting, etc.

In [ ]:
# Step 1: Create Backend for the MCP Server using REST API
print("=" * 70)
print("Step 1: Creating APIM Backend")
print("=" * 70)

import subprocess
import json
import os

# Ensure PATH includes Azure CLI
AZ_PATH = "/opt/homebrew/opt/azure-cli/bin"
env = os.environ.copy()
env["PATH"] = f"{AZ_PATH}:{env.get('PATH', '')}"

# Backend configuration
backend_config = {
    "properties": {
        "url": MCP_BACKEND_URL,
        "protocol": "http",
        "title": "Travel MCP Server",
        "description": "FastMCP Travel Booking Server deployed on Azure Container Apps"
    }
}

# Save to temp file
with open('/tmp/backend.json', 'w') as f:
    json.dump(backend_config, f)

print(f"\nCreating backend 'travel-mcp-backend'...")
print(f"URL: {MCP_BACKEND_URL}")

# Use az rest to call the ARM API directly
backend_cmd = f'''az rest --method PUT \
  --url "https://management.azure.com/subscriptions/{AZURE_SUBSCRIPTION_ID}/resourceGroups/{APIM_RESOURCE_GROUP}/providers/Microsoft.ApiManagement/service/{APIM_NAME}/backends/travel-mcp-backend?api-version=2023-05-01-preview" \
  --body @/tmp/backend.json'''

result = subprocess.run(backend_cmd, shell=True, capture_output=True, text=True, env=env)
if result.returncode == 0:
    print("✅ Backend created successfully!")
else:
    print(f"Result: {result.stderr[:300] if result.stderr else 'OK'}")

print("\n" + "=" * 70)

### Step 2: Create API Definition

We'll create an API that exposes the MCP server through APIM. The API will:
- Listen on `/travel-mcp` path
- Forward requests to the Container App backend
- Support both JSON and SSE responses (required for MCP)

In [ ]:
# Step 2: Create API Definition
print("=" * 70)
print("Step 2: Creating API Definition")
print("=" * 70)

import subprocess
import os

# Ensure PATH includes Azure CLI
AZ_PATH = "/opt/homebrew/opt/azure-cli/bin"
env = os.environ.copy()
env["PATH"] = f"{AZ_PATH}:{env.get('PATH', '')}"

# Create API for MCP server
api_cmd = f'''az apim api create \
  --resource-group {APIM_RESOURCE_GROUP} \
  --service-name {APIM_NAME} \
  --api-id travel-mcp-api \
  --path travel-mcp \
  --display-name "Travel MCP Server" \
  --description "FastMCP Travel Booking Server - Flight search, hotel booking, and currency conversion" \
  --protocols https \
  --service-url {MCP_BACKEND_URL} \
  --subscription-required true'''

print(f"\nCreating API 'travel-mcp-api'...")
print(f"Path: /travel-mcp")

result = subprocess.run(api_cmd, shell=True, capture_output=True, text=True, env=env)
if result.returncode == 0:
    print("✅ API created successfully!")
    print(f"\n   API will be available at: {APIM_GATEWAY_URL}/travel-mcp")
else:
    if "already exists" in result.stderr.lower() or "conflict" in result.stderr.lower():
        print("⚠️  API already exists (this is fine)")
    else:
        print(f"❌ Error: {result.stderr}")

print("\n" + "=" * 70)

### Step 3: Create API Operation for MCP Endpoint

The MCP protocol uses POST requests to `/mcp`. We need to create an operation that:
- Accepts POST to `/mcp`
- Passes through the JSON-RPC body
- Supports SSE responses

In [ ]:
# Step 3: Create API Operation for MCP messages
print("=" * 70)
print("Step 3: Creating MCP Operation")
print("=" * 70)

import subprocess
import os

# Ensure PATH includes Azure CLI
AZ_PATH = "/opt/homebrew/opt/azure-cli/bin"
env = os.environ.copy()
env["PATH"] = f"{AZ_PATH}:{env.get('PATH', '')}"

# Create operation for POST /mcp
operation_cmd = f'''az apim api operation create \
  --resource-group {APIM_RESOURCE_GROUP} \
  --service-name {APIM_NAME} \
  --api-id travel-mcp-api \
  --operation-id mcp-message \
  --display-name "MCP Message" \
  --description "Send MCP protocol messages (initialize, tools/list, tools/call)" \
  --method POST \
  --url-template /mcp'''

print(f"\nCreating operation 'mcp-message'...")
print(f"Method: POST")
print(f"Path: /mcp")

result = subprocess.run(operation_cmd, shell=True, capture_output=True, text=True, env=env)
if result.returncode == 0:
    print("✅ Operation created successfully!")
else:
    if "already exists" in result.stderr.lower() or "conflict" in result.stderr.lower():
        print("⚠️  Operation already exists (this is fine)")
    else:
        print(f"❌ Error: {result.stderr}")

# Also create a catch-all operation for any other paths
print("\nCreating catch-all operation for flexibility...")
catchall_cmd = f'''az apim api operation create \
  --resource-group {APIM_RESOURCE_GROUP} \
  --service-name {APIM_NAME} \
  --api-id travel-mcp-api \
  --operation-id mcp-catchall \
  --display-name "MCP Catch-All" \
  --description "Catch-all for any MCP endpoint" \
  --method POST \
  --url-template "/*"'''

result = subprocess.run(catchall_cmd, shell=True, capture_output=True, text=True, env=env)
if result.returncode == 0:
    print("✅ Catch-all operation created!")
elif "already exists" in result.stderr.lower() or "conflict" in result.stderr.lower():
    print("⚠️  Catch-all already exists (this is fine)")

print("\n" + "=" * 70)

### Step 4: Apply Inbound Policy for MCP

We need a policy that:
1. **Forwards Accept header** - MCP requires `application/json, text/event-stream`
2. **Sets backend** - Routes to our Container App
3. **Handles SSE** - Doesn't buffer streaming responses

In [ ]:
# Step 4: Apply MCP-compatible policy using REST API
print("=" * 70)
print("Step 4: Applying MCP Policy")
print("=" * 70)

import subprocess
import json
import os

# Ensure PATH includes Azure CLI
AZ_PATH = "/opt/homebrew/opt/azure-cli/bin"
env = os.environ.copy()
env["PATH"] = f"{AZ_PATH}:{env.get('PATH', '')}"

# Create policy XML for MCP support
# Note: <base /> in backend section already includes forward-request, so we only customize the backend-id
mcp_policy = '''<policies>
    <inbound>
        <base />
        <set-header name="Accept" exists-action="override">
            <value>application/json, text/event-stream</value>
        </set-header>
        <set-backend-service backend-id="travel-mcp-backend" />
    </inbound>
    <backend>
        <forward-request timeout="120" buffer-response="false" />
    </backend>
    <outbound>
        <base />
        <set-header name="Access-Control-Allow-Origin" exists-action="override">
            <value>*</value>
        </set-header>
    </outbound>
    <on-error>
        <base />
    </on-error>
</policies>'''

print("Policy configured for MCP protocol:")
print("  - Accept header: application/json, text/event-stream")
print("  - Backend: travel-mcp-backend")
print("  - SSE streaming: enabled (buffer-response=false)")
print("  - Timeout: 120 seconds")

# Create policy config
policy_config = {
    "properties": {
        "format": "xml",
        "value": mcp_policy
    }
}

with open('/tmp/policy.json', 'w') as f:
    json.dump(policy_config, f)

# Apply policy to operation using REST API
print("\nApplying policy to mcp-message operation...")
policy_cmd = f'''az rest --method PUT \
  --url "https://management.azure.com/subscriptions/d7713f12-c2af-4980-a889-af285807fddb/resourceGroups/{APIM_RESOURCE_GROUP}/providers/Microsoft.ApiManagement/service/{APIM_NAME}/apis/travel-mcp-api/operations/mcp-message/policies/policy?api-version=2023-05-01-preview" \
  --body @/tmp/policy.json'''

result = subprocess.run(policy_cmd, shell=True, capture_output=True, text=True, env=env)
if result.returncode == 0:
    print("✅ Policy applied successfully!")
else:
    print(f"Result: {result.stderr[:300] if result.stderr else result.stdout[:300]}")

print("\n" + "=" * 70)

## Part 3: Get Subscription Key for Authentication

APIM uses subscription keys for API authentication. Let's get or create a subscription key.

In [ ]:
# Get or create subscription for the API using REST API
print("=" * 70)
print("Getting APIM Subscription Key")
print("=" * 70)

import subprocess
import json
import os

# Ensure PATH includes Azure CLI
AZ_PATH = "/opt/homebrew/opt/azure-cli/bin"
env = os.environ.copy()
env["PATH"] = f"{AZ_PATH}:{env.get('PATH', '')}"

# Create subscription using REST API
print("\nCreating subscription for Travel MCP API...")
subscription_config = {
    "properties": {
        "displayName": "Travel MCP Subscription",
        "scope": f"/subscriptions/{AZURE_SUBSCRIPTION_ID}/resourceGroups/{APIM_RESOURCE_GROUP}/providers/Microsoft.ApiManagement/service/{APIM_NAME}/apis/travel-mcp-api",
        "state": "active"
    }
}

with open('/tmp/subscription.json', 'w') as f:
    json.dump(subscription_config, f)

create_sub_cmd = f'''az rest --method PUT \
  --url "https://management.azure.com/subscriptions/{AZURE_SUBSCRIPTION_ID}/resourceGroups/{APIM_RESOURCE_GROUP}/providers/Microsoft.ApiManagement/service/{APIM_NAME}/subscriptions/travel-mcp-sub?api-version=2023-05-01-preview" \
  --body @/tmp/subscription.json'''

result = subprocess.run(create_sub_cmd, shell=True, capture_output=True, text=True, env=env)
if result.returncode == 0:
    print("✅ Subscription created!")
else:
    if "already exists" in result.stderr.lower() or "conflict" in result.stderr.lower():
        print("⚠️  Subscription already exists")
    else:
        print(f"Note: {result.stderr[:200] if result.stderr else 'Created'}")

# Get the subscription secrets (keys)
print("\nRetrieving subscription key...")
get_keys_cmd = f'''az rest --method POST \
  --url "https://management.azure.com/subscriptions/{AZURE_SUBSCRIPTION_ID}/resourceGroups/{APIM_RESOURCE_GROUP}/providers/Microsoft.ApiManagement/service/{APIM_NAME}/subscriptions/travel-mcp-sub/listSecrets?api-version=2023-05-01-preview"'''

result = subprocess.run(get_keys_cmd, shell=True, capture_output=True, text=True, env=env)
if result.returncode == 0 and result.stdout.strip():
    keys = json.loads(result.stdout)
    SUBSCRIPTION_KEY = keys.get('primaryKey', '')
    if SUBSCRIPTION_KEY:
        print(f"✅ Subscription Key: {SUBSCRIPTION_KEY[:8]}...{SUBSCRIPTION_KEY[-4:]}")
        print(f"\n   Full key (for testing): {SUBSCRIPTION_KEY}")
    else:
        print("❌ No primary key found in response")
        SUBSCRIPTION_KEY = None
else:
    print(f"❌ Could not retrieve subscription key: {result.stderr[:200] if result.stderr else 'Unknown error'}")
    SUBSCRIPTION_KEY = None

print("\n" + "=" * 70)

## Part 4: Test MCP Server Through APIM Gateway

Now let's test the MCP server through the APIM gateway. The endpoint will be:
```
https://apim-a35tm-aiagents.azure-api.net/travel-mcp/mcp
```

We need to include the subscription key in the header: `Ocp-Apim-Subscription-Key`

In [ ]:
# Test MCP Server through APIM
import aiohttp
import json

# APIM endpoint for MCP
APIM_MCP_ENDPOINT = f"{APIM_GATEWAY_URL}/travel-mcp/mcp"

async def test_mcp_through_apim(subscription_key: str):
    """Test MCP protocol through APIM gateway"""
    
    print("=" * 70)
    print("Testing MCP Server Through APIM Gateway")
    print("=" * 70)
    print(f"\n  Endpoint: {APIM_MCP_ENDPOINT}")
    print(f"  Auth: Ocp-Apim-Subscription-Key")
    
    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json, text/event-stream",
        "Ocp-Apim-Subscription-Key": subscription_key
    }
    
    # Test 1: Initialize
    print("\n[1] Testing MCP Initialize...")
    init_request = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "initialize",
        "params": {
            "protocolVersion": "2024-11-05",
            "capabilities": {},
            "clientInfo": {"name": "apim-test", "version": "1.0"}
        }
    }
    
    async with aiohttp.ClientSession() as session:
        try:
            async with session.post(
                APIM_MCP_ENDPOINT,
                json=init_request,
                headers=headers,
                timeout=aiohttp.ClientTimeout(total=30)
            ) as resp:
                print(f"    Status: {resp.status}")
                
                if resp.status == 200:
                    # Read SSE response
                    text = await resp.text()
                    if "event:" in text:
                        # Parse SSE
                        for line in text.split('\n'):
                            if line.startswith('data:'):
                                data = json.loads(line[5:].strip())
                                if 'result' in data:
                                    server_info = data['result'].get('serverInfo', {})
                                    print(f"    ✅ Connected to: {server_info.get('name', 'Unknown')}")
                                    print(f"       Version: {server_info.get('version', 'N/A')}")
                    else:
                        print(f"    Response: {text[:200]}")
                else:
                    text = await resp.text()
                    print(f"    ❌ Error: {text[:200]}")
                    return
                    
        except Exception as e:
            print(f"    ❌ Error: {e}")
            return
    
    # Test 2: List Tools
    print("\n[2] Testing tools/list...")
    tools_request = {
        "jsonrpc": "2.0",
        "id": 2,
        "method": "tools/list"
    }
    
    async with aiohttp.ClientSession() as session:
        try:
            async with session.post(
                APIM_MCP_ENDPOINT,
                json=tools_request,
                headers=headers,
                timeout=aiohttp.ClientTimeout(total=30)
            ) as resp:
                if resp.status == 200:
                    text = await resp.text()
                    for line in text.split('\n'):
                        if line.startswith('data:'):
                            data = json.loads(line[5:].strip())
                            if 'result' in data:
                                tools = data['result'].get('tools', [])
                                print(f"    ✅ Found {len(tools)} tools:")
                                for tool in tools[:5]:
                                    print(f"       - {tool.get('name')}")
                else:
                    print(f"    Status: {resp.status}")
        except Exception as e:
            print(f"    Error: {e}")
    
    print("\n" + "=" * 70)
    print("SUCCESS: MCP Server accessible through APIM Gateway!")
    print("=" * 70)
    print(f"""
  Use this endpoint in your MCP clients:
  
    URL: {APIM_MCP_ENDPOINT}
    Header: Ocp-Apim-Subscription-Key: {subscription_key[:8]}...
    
  Benefits of using APIM:
    ✅ API Key authentication
    ✅ Rate limiting & throttling
    ✅ Analytics & monitoring
    ✅ Centralized API governance
""")

# Run test with subscription key
if SUBSCRIPTION_KEY:
    await test_mcp_through_apim(SUBSCRIPTION_KEY)
else:
    print("❌ No subscription key available. Please check APIM configuration.")

## Part 5: Using APIM MCP with AI Foundry Agent Service

Now let's use the APIM-protected MCP server with the AI Foundry Agent Service.

In [ ]:
# Using APIM MCP endpoint with Azure AI Agents SDK (McpTool)
# Based on official documentation: https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/model-context-protocol-samples

import os
import time

# Add Azure CLI to PATH for the credential to find it
AZ_CLI_PATH = "/opt/homebrew/opt/azure-cli/bin"
if AZ_CLI_PATH not in os.environ.get("PATH", ""):
    os.environ["PATH"] = f"{AZ_CLI_PATH}:{os.environ.get('PATH', '')}"

from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import (
    ListSortOrder,
    McpTool,
    RequiredMcpToolCall,
    SubmitToolApprovalAction,
    ToolApproval,
)
from dotenv import load_dotenv

load_dotenv()

# Configuration
AZURE_AI_PROJECT_ENDPOINT = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
MODEL_DEPLOYMENT = os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME", "gpt-4.1")

# APIM MCP endpoint - the full path including /mcp
APIM_MCP_ENDPOINT = f"{APIM_GATEWAY_URL}/travel-mcp/mcp"

print("=" * 70)
print("Agent Framework Configuration with APIM MCP (Using McpTool SDK)")
print("=" * 70)
print(f"""
  APIM MCP Endpoint: {APIM_MCP_ENDPOINT}
  Auth Header: Ocp-Apim-Subscription-Key
  AI Project: {AZURE_AI_PROJECT_ENDPOINT}
  Model: {MODEL_DEPLOYMENT}
""")

# Initialize the MCP tool with headers
mcp_tool = McpTool(
    server_label="travel_mcp_apim",
    server_url=APIM_MCP_ENDPOINT,
    allowed_tools=[],  # Allow all tools
)

# Add the APIM subscription key header
mcp_tool.update_headers("Ocp-Apim-Subscription-Key", SUBSCRIPTION_KEY)
# mcp_tool.set_approval_mode("never")  # Uncomment to disable approval

print(f"✅ MCP Tool configured with headers: {list(mcp_tool.headers.keys())}")

# Create the AI Project Client
project_client = AIProjectClient(
    endpoint=AZURE_AI_PROJECT_ENDPOINT,
    credential=DefaultAzureCredential(),
)

print("=" * 70)
print("Testing Agent with APIM-Protected MCP Tools")
print("=" * 70)

with project_client:
    agents_client = project_client.agents
    
    # Create a new agent with MCP tool
    agent = agents_client.create_agent(
        model=MODEL_DEPLOYMENT,
        name="APIM_Travel_Agent",
        instructions="""You are a helpful travel assistant. 
        Use the travel booking tools to help users search for flights,
        find hotels, and convert currencies. Be concise and helpful.""",
        tools=mcp_tool.definitions,
    )
    
    print(f"✅ Agent created: {agent.id}")
    print(f"   MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")
    
    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"✅ Thread created: {thread.id}")
    
    # Create message to thread
    user_query = "What flights are available from Seattle to Tokyo on December 15th?"
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content=user_query,
    )
    print(f"\n[User]: {user_query}")
    
    # Create run with MCP tool resources (includes headers)
    run = agents_client.runs.create(
        thread_id=thread.id, 
        agent_id=agent.id, 
        tool_resources=mcp_tool.resources
    )
    print(f"✅ Run created: {run.id}")
    
    # Process the run, handling tool approvals if needed
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)
        
        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break
            
            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    print(f"  Approving MCP tool call: {tool_call.name}")
                    tool_approvals.append(
                        ToolApproval(
                            tool_call_id=tool_call.id,
                            approve=True,
                            headers=mcp_tool.headers,
                        )
                    )
            
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )
        
        print(f"  Run status: {run.status}")
    
    print(f"\n✅ Run completed with status: {run.status}")
    
    if run.status == "failed":
        print(f"❌ Run failed: {run.last_error}")
    else:
        # Fetch and display messages
        messages = agents_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
        print("\n" + "-" * 50)
        for msg in messages:
            if msg.text_messages:
                last_text = msg.text_messages[-1]
                print(f"[{msg.role.upper()}]: {last_text.text.value}")
        print("-" * 50)
    
    # Cleanup
    agents_client.delete_agent(agent.id)
    print("\n✅ Agent cleaned up")

print("\n" + "=" * 70)
print("SUCCESS: Agent used MCP tools through APIM Gateway!")
print("=" * 70)
print(f"""
Benefits of using APIM for MCP:
  ✅ API Key authentication
  ✅ Rate limiting & throttling  
  ✅ Analytics & monitoring
  ✅ Centralized API governance
  ✅ Enterprise-grade security
""")

## Part 6: Agent Framework Integration (Coming Soon)

The Microsoft Agent Framework provides a higher-level abstraction with `HostedMCPTool` and `AzureAIClient`. 
Integration with APIM-protected MCP servers is currently being developed.

**Note:** Microsoft AI Foundry has recently released significant updates including an **AI Gateway** built 
directly into the platform, which provides native API management capabilities. Future tutorials will cover 
how to leverage these new features for MCP server governance.

For now, the recommended approach for APIM-protected MCP servers is to use the **McpTool from the Azure AI 
Agents SDK** (as shown in Part 5 above), which provides direct control over headers and authentication.

In [ ]:
# Agent Framework integration with APIM MCP (Preview)
# This section demonstrates the HostedMCPTool approach - full APIM support coming soon

import os
from typing import Any

# Ensure Azure CLI is in PATH
AZ_CLI_PATH = "/opt/homebrew/opt/azure-cli/bin"
if AZ_CLI_PATH not in os.environ.get("PATH", ""):
    os.environ["PATH"] = f"{AZ_CLI_PATH}:{os.environ.get('PATH', '')}"

from agent_framework import AgentProtocol, AgentRunResponse, ChatMessage, HostedMCPTool
from agent_framework.azure import AzureAIClient
from azure.identity.aio import AzureCliCredential
from dotenv import load_dotenv

load_dotenv()

# APIM MCP endpoint - full path with /mcp
APIM_MCP_FULL_URL = f"{APIM_GATEWAY_URL}/travel-mcp/mcp"

print("=" * 70)
print("Agent Framework with HostedMCPTool (Preview)")
print("=" * 70)
print(f"""
  APIM MCP URL: {APIM_MCP_FULL_URL}
  
  Status: Agent Framework APIM integration is under development.
  
  For APIM-protected MCP servers, use the McpTool approach from Part 5.
  
  Coming soon:
  - Native HostedMCPTool support for APIM headers
  - Azure AI Gateway integration (built into AI Foundry)
""")
print("=" * 70)

# Example code structure for future reference:
print("""
Example (for future use when APIM support is complete):

```python
async with (
    AzureCliCredential() as credential,
    AzureAIClient(async_credential=credential).create_agent(
        name="TravelAgent",
        instructions="You help with travel planning.",
        tools=HostedMCPTool(
            name="TravelMCP",
            url=APIM_MCP_FULL_URL,
            headers={"Ocp-Apim-Subscription-Key": SUBSCRIPTION_KEY},
            approval_mode="never_require",
        ),
    ) as agent,
):
    result = await agent.run("Search for flights to Paris")
    print(result.text)
```

For now, use the McpTool SDK approach demonstrated in Part 5.
""")

## Summary

In this tutorial, we:

1. **Configured environment variables** - Set up APIM and MCP server configuration in `.env`
2. **Created APIM Backend** - Pointed to the Container App URL
3. **Created API** - Exposed the MCP server at `/travel-mcp/mcp`
4. **Configured MCP-compatible policy** - SSE streaming support, proper headers
5. **Added authentication** - APIM subscription key
6. **Tested through APIM** - Verified initialize and tools/list work
7. **Integrated with AI Agents SDK** - Used McpTool with APIM headers

### Benefits of APIM for MCP Servers

| Feature | Direct Access | Through APIM |
|---------|--------------|--------------|
| Authentication | None/Custom | API Keys, OAuth, Certificates |
| Rate Limiting | None | ✅ Built-in |
| Monitoring | Custom logging | ✅ Azure Monitor integration |
| Caching | None | ✅ Response caching |
| Versioning | Manual | ✅ API versioning |
| Multi-region | Manual | ✅ Built-in |

### Azure AI Gateway (New)

Microsoft AI Foundry has released an **AI Gateway** feature built directly into the platform. 
This provides native API management capabilities for AI services, including:

- Load balancing across model deployments
- Token rate limiting
- Usage tracking and analytics
- Simplified authentication

Future tutorials will explore how to leverage AI Gateway alongside or instead of APIM for MCP governance.

### Next Steps

- **Tutorial 19**: Orchestrating multiple MCP servers with agents
- Explore Azure AI Gateway for AI-native API management
- Add OAuth 2.0 authentication for enhanced security
- Set up Azure Monitor alerts for MCP usage